# Creating plots

## 0) Imports and definitions

In [60]:
# Standard
import pandas as pd
import numpy as np
from datetime import datetime

# Viz
import matplotlib.pyplot as plt
import plotly.express as px

# Other
import zeep
import json
import urllib
import requests
from unidecode import unidecode

## 1) WSDL

In [61]:
wsdl = './FachadaWSSGS.wsdl'
client = zeep.Client(wsdl=wsdl)

In [62]:
long_array_type = client.get_type('ns0:ArrayOfflong')
r = client.service.getValoresSeriesVO(long_array_type([7384]), '01/01/2000', '05/04/2021')
aux = zeep.helpers.serialize_object(r[0]['valores'])

In [63]:
df_vendas = pd.DataFrame(aux)
df_vendas['valor'] = df_vendas['valor'].apply(lambda x: int(x['_value_1']))

In [64]:
df_vendas = df_vendas.rename(columns={'valor': 'vendas'})
df_vendas = df_vendas[['mes', 'ano', 'vendas']]
df_vendas.head()

,mes,ano,vendas
0,1,2000,69984
1,2,2000,82262
2,3,2000,76748
3,4,2000,91613
4,5,2000,98154


In [65]:
df_vendas['ano'].value_counts()

2000    12
2001    12
2020    12
2019    12
2018    12
2017    12
2016    12
2015    12
2014    12
2013    12
2012    12
2011    12
2010    12
2009    12
2008    12
2007    12
2006    12
2005    12
2004    12
2003    12
2002    12
2021     4
Name: ano, dtype: int64

## 3) Data POA 

In [66]:
url = 'https://dadosabertos.poa.br/api/3/action/datastore_search?resource_id=b56f8123-716a-4893-9348-23945f1ea1b9'

r = requests.get(url)
data_json = r.json()

In [67]:
df_acidentes = pd.DataFrame.from_dict(data_json['result']['records'])

,_id,data_extracao,predial1,queda_arr,data,feridos,feridos_gr,mortes,morte_post,fatais,...,longitude,latitude,log1,log2,tipo_acid,dia_sem,hora,noite_dia,regiao,consorcio
0,1,2021-09-01T01:33:40,0,0,2020-10-17T00:00:00,1,0,0,0,0,...,0.000000,0.000000,R MARCOS MOREIRA,R GASTON ENGLERT,ABALROAMENTO,SÁBADO,19:00:00.0000000,NOITE,NORTE,
1,2,2021-09-01T01:33:40,8487,0,2016-01-01T00:00:00,0,0,0,1,1,...,-51.095216,-30.134588,ESTR JOAO DE OLIVEIRA REMIAO,,CHOQUE,SEXTA-FEIRA,12:00:00.0000000,DIA,LESTE,
2,3,2021-09-01T01:33:40,0,0,2016-01-02T00:00:00,1,0,0,0,0,...,-51.200627,-30.004450,AV EDVALDO PEREIRA PAIVA,,QUEDA,SÁBADO,06:30:00.0000000,DIA,NORTE,
3,4,2021-09-01T01:33:40,240,0,2016-01-02T00:00:00,1,0,0,0,0,...,-51.218421,-30.046960,AV ERICO VERISSIMO,,ABALROAMENTO,SÁBADO,18:30:00.0000000,NOITE,CENTRO,STS
4,5,2021-09-01T01:33:40,2084,0,2016-01-02T00:00:00,3,0,0,0,0,...,-51.129850,-30.093210,ESTR JOAO DE OLIVEIRA REMIAO,,CHOQUE,SÁBADO,20:15:00.0000000,NOITE,LESTE,


In [68]:
df_acidentes['mes'] = df_acidentes['data'].apply(lambda x: pd.to_datetime(x).month)
df_acidentes['ano'] = df_acidentes['data'].apply(lambda x: pd.to_datetime(x).year)
df_acidentes['tipo_acid'] = df_acidentes['tipo_acid'].apply(lambda x: unidecode(x.lower()))

In [74]:
df_acidentes = df_acidentes[['mes', 'ano', 'tipo_acid', 'feridos', 'fatais', 'caminhao', 'moto', 'latitude', 'longitude']]
df_acidentes

,mes,ano,tipo_acid,feridos,fatais,caminhao,moto,latitude,longitude
0,10,2020,abalroamento,1,0,0,1,0.000000,0.000000
1,1,2016,choque,0,1,0,0,-30.134588,-51.095216
2,1,2016,queda,1,0,0,1,-30.004450,-51.200627
3,1,2016,abalroamento,1,0,0,0,-30.046960,-51.218421
4,1,2016,choque,3,0,0,0,-30.093210,-51.129850
...,...,...,...,...,...,...,...,...,...
95,1,2016,abalroamento,0,0,0,0,NaN,NaN
96,1,2016,colisao,0,0,0,0,-30.036384,-51.240630
97,1,2016,abalroamento,0,0,1,0,-30.036384,-51.240630
98,1,2016,choque,0,0,0,0,-30.029305,-51.215888


## 4) Auxiliary functions

## 5) Plots

### 5.1) Acidentes

In [71]:
df = df_acidentes.copy()

In [72]:
cat_list = ['mortes']
cat_values = [0]

df_aux = df.copy()
for col, value, in zip(bool_list, bool_values):
    print(col, value)
    df_aux = df_aux[df_aux[col] == value]

mortes 0


KeyError: 'mortes'

In [ ]:
df_aux.columns

In [ ]:
df_acidentes['tipo_acid'].value_counts()

In [ ]:
df_plot = df_aux.copy()

In [ ]:
fig = px.histogram(df_plot, x="total_bill", y="tip", color="sex", marginal="rug", hover_data=df.columns)
fig.show()